In [1]:
source("tools.R")
library('DescTools')

Warning message:
"package 'plyr' was built under R version 3.3.3"Warning message:
"package 'tidyverse' was built under R version 3.3.3"Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Warning message:
"package 'ggplot2' was built under R version 3.3.3"Warning message:
"package 'tibble' was built under R version 3.3.3"Warning message:
"package 'tidyr' was built under R version 3.3.3"Warning message:
"package 'readr' was built under R version 3.3.3"Warning message:
"package 'purrr' was built under R version 3.3.3"Warning message:
"package 'dplyr' was built under R version 3.3.3"Conflicts with tidy packages ---------------------------------------------------
arrange():   dplyr, plyr
compact():   purrr, plyr
count():     dplyr, plyr
failwith():  dplyr, plyr
filter():    dplyr, stats
id():        dplyr, plyr
lag():       dplyr, stats
mutate():    dplyr, plyr
rename():    dplyr, plyr
summa

# Plots

## Real-world plot

In [2]:
data <- read_csv("data/real-world/to-plot.csv")

kernels <- c("l", "p", "r", "l+p", "l+r", "p+r", "l*r", "l*p", "p*r", "l+r+p", "l+r*p", "l*r+p", "l*p+r", "l*r*p")

lmls_prior_f <- data

Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)


In [3]:
plot_lmls <- function(prop_data, title, hide_x=FALSE, hide_y=FALSE) {
    plot <- prop_data %>%
                ggplot(aes(x=kernel, y=value)) + 
                    geom_bar(stat="identity") +
                    coord_cartesian (ylim=c(0, 1)) +
                    labs(title = title) +
                    ggthemes::theme_few() +
                    xlab("Kernel composition") + 
                    ylab("Mean likelihood (scaled)") +
                    #geom_errorbar(aes(ymin=as.numeric(lo_ci), ymax=as.numeric(hi_ci)),
                    #                  width=.3,                   
                    #                  position=position_dodge(.9)) +
                    theme(axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          text = element_text(size=12, family="serif"),
                          plot.title = element_text(hjust = 0.5))
    
    if(hide_x){
        plot <- plot + theme(axis.title.x=element_blank())
    }
    if(hide_y){
        plot <- plot + theme(axis.title.y=element_blank())
    }
    
    return(plot)
}

In [4]:
to_plot <- lmls_prior_f %>%
            group_by(kernel, scenario) %>%
            summarize(value = mean(lml))

# Remove single components
to_plot <- to_plot %>%
                filter(kernel != 'l', kernel != 'p', kernel != 'r')

# Scaling the 'data to plot' to a 0-1 range
min_v = min(to_plot$value)
range_v = max(to_plot$value) - min(to_plot$value)

to_plot <- to_plot %>%
                group_by( scenario ) %>%
                mutate(value = (value - min(value)) / (max(value) - min(value)))
# Plotting magic
to_plot$kernel <- factor(to_plot$kernel, levels=kernels)

lmls_temperature <- to_plot %>% filter(scenario == "Temperature")
lmls_rain <- to_plot %>% filter(scenario == "Rain")
lmls_sales <- to_plot %>% filter(scenario == "Sales")
lmls_gym <- to_plot %>% filter(scenario == "Gym members")
lmls_salary <- to_plot %>% filter(scenario == "Salary")
lmls_fb <- to_plot %>% filter(scenario == "FB Friends")

p1 <- plot_lmls(lmls_temperature, "Temperature", hide_x=TRUE)
p2 <- plot_lmls(lmls_rain, "Rain", hide_x=TRUE)
p3 <- plot_lmls(lmls_sales, "Sales", hide_x=TRUE, hide_y=TRUE)
p4 <- plot_lmls(lmls_gym, "Gym members", hide_y=TRUE, hide_x=TRUE)
p5 <- plot_lmls(lmls_salary, "Salary", hide_x=TRUE, hide_y=TRUE)
p6 <- plot_lmls(lmls_fb, "FB Friends", hide_y=TRUE, hide_x=TRUE)

pdf("Images/paper_images/real-world.pdf", width=8, height=4)
multiplot(p1, p2, p3, p4, p5, p6, cols=3)
dev.off()

Warning message:
"package 'bindrcpp' was built under R version 3.3.3"

png 
  2

# Only values

In [62]:
data %>% 
    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
    group_by(scenario) %>%
    summarize(kernel[which.max(lml)], lml[which.max(lml)])

scenario,kernel[which.max(lml)],lml[which.max(lml)]
FB Friends,l+r+p,3532.55614
Gym members,l+r*p,104.17351
Rain,l+r*p,-188.08552
Salary,p+r,-885.91397
Sales,p+r,-53.80874
Temperature,p*r,423.88905


In [63]:
data %>% 
    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
    filter(scenario=="Sales")

scenario,kernel,lml
Sales,l+r,-158.81582
Sales,l+p,-1051.41186
Sales,p+r,-53.80874
Sales,l*r,-1129.74901
Sales,l*p,-1489.35883
Sales,p*r,-97.92928
Sales,l+r+p,-1046.57994
Sales,l+r*p,-60.04926
Sales,l*r+p,-1110.02837
Sales,l*p+r,-168171.80759
